In [1]:
import torch
from models.stylegan2.model import Generator  # Assuming this is the path to the Generator class
import numpy as np
from utils import common


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [3]:
torch.cuda.empty_cache()

In [4]:
project_path = '/home/melih.cosgun/GENERATIVE'

In [5]:
# Load the model
device = "cuda:1" 
stylegan_model_path = "pretrained_models/stylegan2-ffhq-config-f.pt" 

stylegan_model = Generator(1024, 512, 8).to(device)

state_dict = torch.load(stylegan_model_path)
face_pool = torch.nn.AdaptiveAvgPool2d((256, 256))
stylegan_model.load_state_dict(state_dict['g_ema'], strict=False)

stylegan_model.eval()

Generator(
  (style): Sequential(
    (0): PixelNorm()
    (1): EqualLinear(512, 512)
    (2): EqualLinear(512, 512)
    (3): EqualLinear(512, 512)
    (4): EqualLinear(512, 512)
    (5): EqualLinear(512, 512)
    (6): EqualLinear(512, 512)
    (7): EqualLinear(512, 512)
    (8): EqualLinear(512, 512)
  )
  (input): ConstantInput()
  (conv1): StyledConv(
    (conv): ModulatedConv2d(512, 512, 3, upsample=False, downsample=False)
    (noise): NoiseInjection()
    (activate): FusedLeakyReLU()
  )
  (to_rgb1): ToRGB(
    (conv): ModulatedConv2d(512, 3, 1, upsample=False, downsample=False)
  )
  (convs): ModuleList(
    (0): StyledConv(
      (conv): ModulatedConv2d(512, 512, 3, upsample=True, downsample=False)
      (noise): NoiseInjection()
      (activate): FusedLeakyReLU()
    )
    (1): StyledConv(
      (conv): ModulatedConv2d(512, 512, 3, upsample=False, downsample=False)
      (noise): NoiseInjection()
      (activate): FusedLeakyReLU()
    )
    (2): StyledConv(
      (conv): Modul

In [6]:
# Random Generation

latent_z = np.random.randn(1, 512).astype("float32")
latent_z = torch.from_numpy(latent_z).to(device)

img, w_plus = stylegan_model(styles=[latent_z], input_is_latent=False, randomize_noise=False, return_latents=True)   # Adapt this line according to the actual function

img = face_pool(img)

img = common.tensor2im(img[0])

resize_amount = (256, 256)
img.resize(resize_amount)

img.save(f'{project_path}/test_images/test.png')

w_plus = w_plus[0].detach().cpu().numpy()

# Save the latent code
np.save(f'{project_path}/test_latents/test.npy' , w_plus)


In [7]:
import torchvision.transforms as transforms

EXPERIMENT_DATA_ARGS = {
    "ffhq_encode": {
        "model_path": "pretrained_models/model_rgb_loss.pt", # changed to custom model
        "image_path": "/home/melih.cosgun/GENERATIVE/test_inputs/input_img.png",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS['ffhq_encode']

In [8]:
from models.psp import pSp
from argparse import Namespace

model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')

opts = ckpt['opts']

opts['device'] = device

opts['checkpoint_path'] = model_path
if 'learn_in_w' not in opts:
    opts['learn_in_w'] = False
if 'output_size' not in opts:
    opts['output_size'] = 1024

opts = Namespace(**opts)
net = pSp(opts)
net.eval()

net = net.to(device)
#net.cuda()

Loading pSp from checkpoint: pretrained_models/model_rgb_loss.pt


RuntimeError: Error(s) in loading state_dict for GradualStyleEncoder:
	Missing key(s) in state_dict: "condition_layer.conv1.weight", "condition_layer.conv1.bias", "condition_layer.conv2.weight", "condition_layer.conv2.bias", "condition_layer.conv3.weight", "condition_layer.conv3.bias", "condition_layer.bn1.weight", "condition_layer.bn1.bias", "condition_layer.bn1.running_mean", "condition_layer.bn1.running_var", "condition_layer.bn2.weight", "condition_layer.bn2.bias", "condition_layer.bn2.running_mean", "condition_layer.bn2.running_var". 

In [8]:
import dlib
from scripts.align_all_parallel import align_face
def run_alignment(image_path):
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor)
  print("Aligned image has shape: {}".format(aligned_image.size))
  return aligned_image

In [9]:
image_path = f'{project_path}/test_inputs/input_img.png'

input_image = run_alignment(image_path)

input_image.resize((256, 256))

img_transforms = EXPERIMENT_ARGS['transform']

transformed_image = img_transforms(input_image)

Aligned image has shape: (256, 256)


In [10]:
with torch.no_grad():
    input = transformed_image.unsqueeze(0)
    result_image, w_plus = net(input.to(device).float(), randomize_noise=False, return_latents=True)

In [16]:
output_image = common.tensor2im(result_image[0])
output_image.save(f'{project_path}/test_images/test.png')
w_plus = w_plus[0].detach().cpu().numpy()
np.save(f'{project_path}/test_latents/test.npy' , w_plus)

In [29]:
original_latents = np.load(f'{project_path}/test_latents/test.npy')

In [30]:
original_latents.shape

(18, 512)

In [31]:
modification_list = ['Bald', 'Black_Hair', 'Blond_Hair', 'Eyeglasses', 'Goatee', 'Male', 'Mustache', 'Smiling']

for modification in modification_list:
    print(f'Running for modification : {modification}')
    modifying_latent = np.load(f'{project_path}/svm/1.0/{modification}.npy')
    modification_experiment_path = os.path.join(f'{project_path}/output/', modification)
    os.makedirs(modification_experiment_path, exist_ok=True)

    modifying_latent = modifying_latent.reshape(18, 512)
    modifying_latent = modifying_latent.astype(np.float32)

    for i in range(0, 50, 5):
        edited_latents = original_latents + i * modifying_latent
        edited_latents = torch.from_numpy(edited_latents).to(device)
#img, _ = stylegan_model(styles=[edited_latents[0].unsqueeze(0)], input_is_latent=True, randomize_noise=False, return_latents=False) 
        img, _ = stylegan_model(styles=[edited_latents.unsqueeze(0)], input_is_latent=True, randomize_noise=False, return_latents=False) 

        img = face_pool(img)

        img = common.tensor2im(img[0])

        resize_amount = (256, 256)
        img.resize(resize_amount)

        img.save(os.path.join(modification_experiment_path, f'{i}.png'))

Running for modification : Bald
Running for modification : Black_Hair
Running for modification : Blond_Hair
Running for modification : Eyeglasses
Running for modification : Goatee
Running for modification : Male
Running for modification : Mustache
Running for modification : Smiling
